- split the data into a training and validation set (or apply cross validation)

- conduct an exploratory data analysis

- train a regression model

- iteratively optimize the model by expanding or selecting features

- regularize the model to avoid overfitting

- calculate a RMSLE for the training and validation set

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import numpy as np


In [2]:

# Import the relevant sklearn packages
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score  


In [3]:

# Set the figure size of matplotlib plots to (12,6)
plt.rcParams['figure.figsize'] = (28,14)

In [4]:
df = pd.read_csv('data/train.csv', index_col=0, parse_dates = True)
df.head()

#sns.lineplot(data=df, x=df.index, y='count')

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
datetime,,,,,,,,,,,
2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


datetime: date and hour in "mm/dd/yyyy hh:mm" format

season: Four categories-> 1 = spring, 2 = summer, 3 = fall, 4 = winter

holiday: whether the day is a holiday or not (1/0)

workingday: whether the day is neither a weekend nor holiday (1/0)

weather: Four Categories of weather

        1-> Clear, Few clouds, Partly cloudy
        2-> Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
        3-> Light Snow and Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
        4-> Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog

temp: hourly temperature in Celsius

atemp: "feels like" temperature in Celsius

humidity: relative humidity

windspeed: wind speed

casual + registered = count

In [5]:
# df.insert(0, 'day_of_week', df.index.dayofweek)
df = df.drop(['casual','registered'], axis=1)
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10886 entries, 2011-01-01 00:00:00 to 2012-12-19 23:00:00
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   atemp       10886 non-null  float64
 6   humidity    10886 non-null  int64  
 7   windspeed   10886 non-null  float64
 8   count       10886 non-null  int64  
dtypes: float64(3), int64(6)
memory usage: 850.5 KB


In [6]:
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year

df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10886 entries, 2011-01-01 00:00:00 to 2012-12-19 23:00:00
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      10886 non-null  int64  
 1   holiday     10886 non-null  int64  
 2   workingday  10886 non-null  int64  
 3   weather     10886 non-null  int64  
 4   temp        10886 non-null  float64
 5   atemp       10886 non-null  float64
 6   humidity    10886 non-null  int64  
 7   windspeed   10886 non-null  float64
 8   count       10886 non-null  int64  
 9   hour        10886 non-null  int64  
 10  day         10886 non-null  int64  
 11  month       10886 non-null  int64  
 12  year        10886 non-null  int64  
dtypes: float64(3), int64(10)
memory usage: 1.2 MB


In [7]:
#change int columns to cartegory
cat_cols = ['season', 'holiday', 'workingday',
            'weather', 'hour', 'day', 'month','year']
for col in cat_cols:
    df[col] = df[col].astype('category')
df.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 10886 entries, 2011-01-01 00:00:00 to 2012-12-19 23:00:00
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   season      10886 non-null  category
 1   holiday     10886 non-null  category
 2   workingday  10886 non-null  category
 3   weather     10886 non-null  category
 4   temp        10886 non-null  float64 
 5   atemp       10886 non-null  float64 
 6   humidity    10886 non-null  int64   
 7   windspeed   10886 non-null  float64 
 8   count       10886 non-null  int64   
 9   hour        10886 non-null  category
 10  day         10886 non-null  category
 11  month       10886 non-null  category
 12  year        10886 non-null  category
dtypes: category(8), float64(3), int64(2)
memory usage: 597.9 KB


In [8]:
df.describe()


,temp,atemp,humidity,windspeed,count
count,10886.00000,10886.000000,10886.000000,10886.000000,10886.000000
mean,20.23086,23.655084,61.886460,12.799395,191.574132
std,7.79159,8.474601,19.245033,8.164537,181.144454
min,0.82000,0.760000,0.000000,0.000000,1.000000
25%,13.94000,16.665000,47.000000,7.001500,42.000000
50%,20.50000,24.240000,62.000000,12.998000,145.000000
75%,26.24000,31.060000,77.000000,16.997900,284.000000
max,41.00000,45.455000,100.000000,56.996900,977.000000


In [12]:
### Split data ###
X = df.drop(columns=['count'])
y = df['count']

X_train, X_test, y_train, y_test =  train_test_split(X, y, random_state = 42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape


((8164, 12), (2722, 12), (8164,), (2722,))

In [ ]:
ax = plt.figure(figsize=(28, 14))

pearsoncorr = df.corr(method='pearson')
sns.heatmap(pearsoncorr, xticklabels=pearsoncorr.columns, yticklabels=pearsoncorr.columns,
            cmap='Greens', annot=True)



In [ ]:
del df['temp']
X = df.iloc[:,:-1]
y = df.iloc[:,-1]
X 

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:

rf = RandomForestRegressor(max_depth=5, random_state=0)

In [ ]:

rf.fit(X_train, y_train)

In [ ]:

pd.DataFrame({'importance': rf.feature_importances_, 'feature': X_train.columns}).\
sort_values('importance', ascending=False)  #feature_importance- use for RF model

In [ ]:

from sklearn.linear_model import Lasso

In [ ]:

l = Lasso()

In [ ]:

l.fit(X_train, y_train)

In [ ]:

pd.DataFrame({'feature': X_train.columns, 'coefficient': np.abs(l.coef_)}).sort_values('coefficient', ascending=False)

In [ ]:
m = LinearRegression()
m.fit(X_train,y_train)
ypred = m.predict(X_test)


In [ ]:

m.score(X_train, y_train)


In [ ]:
m.score(X_test, y_test)




In [ ]:
def RMSLE(y_train,y_pred):
    return np.sqrt(mean_squared_log_error(y_train, y_pred))

In [ ]:
RMSLE(y_train, ypred)

In [ ]:
To optimize your model against the RMSLE, you should take the logarithm of the target colum (y). Because 0 is a valid target value, use the log of 𝑦+1 instead:

ylog = np.log1p(y)
Then train your model on the transformed column ylog. To bring back your log predictions to the original scale you have to apply the inverse transformation on the predictions:

ypred = np.exp(ypredlog)-1
You can then calculate the RMSLE score using sklearn:

from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error(y, ypred))

In [ ]:
fig, axes = plt.subplots(3,1, figsize = (20,12))

sns.countplot(train["season"], ax = axes[0], palette="Set1")
sns.countplot(train["weather"], ax = axes[1], palette="Set1")
sns.countplot(train["windspeed"], ax = axes[2])
plt.xticks(rotation = 60, )

In [ ]:
_, axes = plt.subplots(1,1, figsize = (20,12))
sns.boxplot(x=train["hour"], y=train["count"])

In [ ]:
df['ideal'] = df[['temp', 'windspeed']].apply(lambda x: (0, 1)[x['temp'] > 27 and x['windspeed'] < 30], axis = 1)
df['sticky'] = df[['humidity', 'workingday']].apply(lambda x: (0, 1)[x['workingday'] == 1 and x['humidity'] >= 60], axis = 1)